# In The Name Of ALLAH

# Twitter Sentiment Analysis Using Logistic Regression

# Mohammad Hossein Amini

In this project, we will do sentiment analysis on some tweets using **Logistic Regression**.

After preprocessing each tweet, we'll count frequency of its words in both positive and negative tweets. So for each tweet, there are 2 features: Positive and Negative frequencies. we'll use these feature vectors to classify tweets!

we would implement logistic regression using **keras**.

In [ ]:
import nltk                                  
from nltk.corpus import twitter_samples      
import matplotlib.pyplot as plt              
import numpy as np       
import keras
from utils import process_tweet, build_freqs

Let's download the dataset.

In [ ]:
nltk.download('twitter_samples')
nltk.download('stopwords')

Now we would create some lists containing our tweets as strings.

In [ ]:
# select the lists of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# concatenate the lists, 1st part is the positive tweets followed by the negative
tweets = all_positive_tweets + all_negative_tweets

# let's see how many tweets we have
print("Number of tweets: ", len(tweets))

In [ ]:
labels = np.append(np.ones((len(all_positive_tweets))), np.zeros((len(all_negative_tweets))))

Let's count frequency of each word in both positive and negative tweets. We'd use ```build_freqs``` function for this purpose.

In [ ]:
# create frequency dictionary
freqs = build_freqs(tweets, labels)

# check data type
print(f'type(freqs) = {type(freqs)}')

# check length of the dictionary
print(f'len(freqs) = {len(freqs)}')

Time for preprocessing and extracting features! ```extractFeatures``` would get a tweet and returns the 2-element feature vector.

In [ ]:
def extractFeatures(tweet):
  tweet = process_tweet(tweet)
  pos, neg = 0, 0
  for word in tweet:
    pos += freqs.get((word, 1.), 0)
    neg += freqs.get((word, 0.), 0)
  feature_vec = np.array([pos, neg])
  return feature_vec

In [ ]:
index = 15
print(tweets[index])
print(extractFeatures(tweets[index]))

Splitting data into train and test now.

In [ ]:
train_tweets = all_positive_tweets[:4000] + all_negative_tweets[:4000]
train_labels = [1. for i in range(4000)] + [0. for i in range(4000)]
test_tweets = all_positive_tweets[4000:] + all_negative_tweets[4000:]
test_labels = [1. for i in range(1000)] + [0. for i in range(1000)]

In [ ]:
print(len(train_tweets), len(test_tweets))
print(len(train_labels), len(test_labels))

We'd have some numpy arrays to keep training set and test set.

In [ ]:
X_train = np.zeros((len(train_tweets), 2))
y_train = np.array(train_labels)
X_test = np.zeros((len(test_tweets), 2))
y_test = np.array(test_labels)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
for i in range(X_train.shape[0]):
  X_train[i] = extractFeatures(train_tweets[i])

for i in range(X_test.shape[0]):
  X_test[i] = extractFeatures(test_tweets[i])

In [ ]:
print(X_train[15], y_train[15])
print(X_train[5])

Let's do the logistic regression now!

In [ ]:
model = keras.Sequential([keras.layers.Dense(1, activation='sigmoid', input_shape=(2,))])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

In [ ]:
print([model.weights[i].numpy() for i in range(2)])

Testing and seeing our brilliant result would be so fun.

In [ ]:
index = 1500
print(model.predict(X_test[index:index+1, :]))

And finally, we would create a ```classify``` function which gets a string and classifies it!

In [ ]:
def classify(tweet):
  tweet = extractFeatures(tweet)
  return model.predict(tweet[np.newaxis, :])

In [ ]:
neg = 'I hated it!'
pos = 'It was an honor to view this great one!'
print(classify(neg), classify(pos))